### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [8]:
print("콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.")

콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.


In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [10]:
import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

pdf_path = '..\data\콘텐츠분쟁해결_사례.pdf'

if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"{pdf_path}라는 파일이 존재하지 않습니다. ")

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_12456\3309375398.py:7: SyntaxWarning: invalid escape sequence '\d'
  pdf_path = '..\data\콘텐츠분쟁해결_사례.pdf'


In [13]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

try: 
    # PyPDFLoader 인스턴스 객체를 반환해서 loader 변수의 메모리에 할당
    loader = PyPDFLoader(pdf_path)
    # print(f"type(loader): {type(loader)}") # type(loader): <class 'langchain_community.document_loaders.pdf.PyPDFLoader'>
    
    # loader 인스턴스를 Document 객체 리스트로 변환
    docs = loader.load()
    # print(f"type(docs): {type(docs)}") # type(docs): <class 'list'>
    # print(f"문서 타입: {type(docs[0])}") # 문서 타입: <class 'langchain_core.documents.base.Document'>
    
    # 첫 번째 문서의 메타데이터 출력하기
    # print(json.dumps(docs[0].metadata, indent=4, ensure_ascii=False))
    
    # docs의 길이
    # print(f"len(docs): {len(docs)}") # len(docs): 109
    
    # 한 번 출력해보기
    # if docs[10]:
    #     pprint(docs[10])
    
    # 1단계: 문서 분할 설정
    # 1-1. 분할기 생성
    pythontext_splitter = RecursiveCharacterTextSplitter(   
        chunk_size = 1500,      # 청크 크기 (최대 길이): 1500자
        chunk_overlap=300,      # 맥락 보존을 위해 중복 단어 300개까지 허용
        separators=[
            "\n【사건개요】",     # 법률 문서 섹션 구분자
            "\n【쟁점사항】",     # 쟁점 부분 구분
            "\n【처리경위】",     # 처리 과정 구분
            "\n【처리결과】",     # 결과 부분 구분
            "\n■", "\n\n", "\n", ".", " ", "" # 자연스러운 분할을 위해 추가
        ]
    )
    # 1-2. 분할기 적용
    chunks = pythontext_splitter.split_documents(documents=docs)
    
    # 2단계: 임베딩 모델 설정
    python_embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large", # 고성능 임베딩 모델로 설정
        
        # 고성능 임베딩 벡터 차원(20000차원) 
        # "Invalid value for 'dimensions' = 20000. Must be less than or equal to 3072."
        # 최대 3072 차원까지 가능
        # dimensions=20000
        dimensions=3072 # 최대 3072 차원까지 가능해서 최대 차원(3072)으로 설정
    )
    
    # 저장 단계: 저장하기
        # documents 매개변수에 입력 된 chunks의 텍스트를 
        # embedding에 입력 된 python_embeddings 모델을 사용하여 
        # 컴퓨터가 읽을 수 있도록 3072차원의 숫자 배열(임베딩 백터)로 변환 후
        # FAISS라는 검색 속도 향상 클래스로 더욱 빠르게 찾을 수 있도록 구조화 후
        # vector_store 변수의 메모리에 저장
    vector_store = FAISS.from_documents(documents=chunks, embedding=python_embeddings)
    
    # 3단계: 검색기 설정
    python_retriever = vector_store.as_retriever(
        # search_type="similarity", # search_type의 default 값인 "similarity" 말고
        search_type="mmr",  # 다양성을 고려하여 mmr 알고리즘 적용
        search_kwargs={
            "k": 5,         # 테스트 객체 중 가장 관련성이 높고 다양한 상위 5개의 Document 객체 반환
            "fetch_k":30    # 30개의 Document 객체를 테스트 가져오고
            }
    )
    
    # 4단계: LLM 설정
    python_llm = ChatOpenAI(
        model="gpt-4o",     # 모델은 gpt-4o
        temperature=0,      # 창의적이게 하지 말고
        max_tokens=1500     # 충분한 답변 길이를 반환해라
    )
    
    # 5단계: 법률 자문 프롬프트 작성
    pythonprompt_template = """
        당신은 콘텐츠 분야 전문 법률 자문사입니다. 
        아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

        관련 분쟁사례:
        {context}

        상담 내용: {question}

        답변 가이드라인:
        1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
        2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
        3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
        4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
        5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

        
        
        전문 법률 조언:
        """
    # 법률 자문 프롬프트 템플릿 생성
    prompt = PromptTemplate(
        template=pythonprompt_template,
        input_variables=["context", "question"]
    )

    # 6단계: QA 체인 생성
    python_qa_chain = RetrievalQA.from_chain_type(
        llm=python_llm,     # python_llm으로 정의한 모델 사용
        retriever=python_retriever, # 리트리버가 가져온 모든 문서
        chain_type="stuff", # 리트리버가 가져온 모든 문서를 그대로 llm 모델에 전달
        chain_type_kwargs={"prompt": prompt},   # 프롬프트 입력
        return_source_documents=True    # 참조 문서도 반환
    )
    
    # 7단계: 테스트 질문 작성
    pythontest_questions = [
        "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
        "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
        "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
        "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
        "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
    ]

    # 8단계: 분쟁 유형 분류 함수
    def classify_dispute_type(query):
        game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
        elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
        web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
        
        query_lower = query.lower()
        
        if any(keyword in query_lower for keyword in game_keywords):
            return "게임"
        elif any(keyword in query_lower for keyword in elearning_keywords):
            return "이러닝"
        elif any(keyword in query_lower for keyword in web_keywords):
            return "웹콘텐츠"
        else:
            return "기타"
        


    for q_num, question in enumerate(pythontest_questions, 1):
        
        print(f"질문 {q_num}. ")
        print(f"{question}")
        print("-"*100)    

        dispute_type = classify_dispute_type(question) # 질문 유형 분류
        
        # 유형에 따른 추가 정보 제공
        if dispute_type == "게임":
            print(f"분쟁 유형: {dispute_type}\t\t\t게임 관련 분쟁 유형은 게임사에 문의하시길 바랍니다. ")
        elif dispute_type == "이러닝":
            print(f"분쟁 유형: {dispute_type}\t\t\t이러닝 관련 분쟁 유형은 이러닝 강의를 진행하고 있는 학원사에 문의하시길 바랍니다. ")
        elif dispute_type == "웹콘텐츠":
            print(f"분쟁 유형: {dispute_type}\t\t\t웹 콘텐츠 관련 분쟁 유형은 해당 웹 콘텐츠를 개발하고 있는 개발사에 문의하시길 바랍니다. ")
        else: 
            print(f"분쟁 유형: {dispute_type}\t\t\t기타 분쟁 유형은 각각의 알맞은 곳에 문의하시길 바랍니다. ")
        
        print("-"*100)
        result = python_qa_chain.invoke({"query": question})
        answer = result["result"]
        source_docs = result["source_documents"]
        
        print("답변: ")
        print(f"{answer}")
        print("-"*100)    
        
        print("\t참조 문서: ")
        for doc_num, doc in enumerate(source_docs[:3], 1):
            print("-"*100)    
            # 각 Document 객체의 metadata 필드에서 'page'값을 가져오고 없으면 'N/A'로 설정
            page = doc.metadata.get('page', 'N/A')
            # 각 Document 객체의 page_content에서 처음 100글자를 가져와 줄바꿈을 공백으로 대체해서 미리보기로 저장
            preview = doc.page_content[:100].replace('\n', '\t')
            # 출력
            print(f"\t\t{doc_num}. 페이지 {page}: {preview}...")


        print("="*100)
    
except Exception as e:
    print(f"에러 발생: {e}")

질문 1. 
온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
----------------------------------------------------------------------------------------------------
분쟁 유형: 게임			게임 관련 분쟁 유형은 게임사에 문의하시길 바랍니다. 
----------------------------------------------------------------------------------------------------
답변: 
전문 법률 조언:

1. **사례 기반 답변**:
   - 제시된 사례 중 "시스템 오류로 소멸된 아이템 복구 요구" 사례를 참고할 수 있습니다. 이 사례에서는 신청인이 시스템 오류로 인해 소멸된 아이템의 복구를 요구했으나, 계정 명의자가 아니어서 복구가 거부된 상황이 있었습니다. 이 경우, 계정 명의자가 직접 복구 요청을 해야 한다는 점이 강조되었습니다.

2. **법적 근거 제시**:
   - 관련 법령으로는 "약관의 규제에 관한 법률" 제11조가 있으며, 이는 고객의 권익 보호를 목적으로 합니다. 또한, "소비자분쟁해결기준" (공정위고시 제2010-1호)도 참고할 수 있습니다.

3. **판례 참조**:
   - 비슷한 사례로는 "2009_시스템 오류로 인한 손실 아이템 복구 요구"가 있습니다. 이 사례에서는 시스템 오류로 인한 손실에 대해 복구를 요구했으나, 계정 명의자가 아니면 복구가 불가하다는 점이 확인되었습니다.

4. **실무 가이드**:
   - **1단계**: 계정 명의 확인 - 본인이 계정 명의자가 아니라면, 명의자에게 복구 요청을 하도록 안내합니다.
   - **2단계**: 게임 회사의 고객센터에 공식적으로 복구 요청을 합니다. 이때, 시스템 오류로 인한 손실임을 명확히 설명하고, 관련 증거(예: 오류 발생 시점의 스크린샷 등)를 제출합니다.
   - **3단계**: 게임 